#### El cliente nos ha solicitado realizar algunos métodos que resuelvan consultas específicas sobre las tablas movies_df, ratings_df y tags_df. Lee cuidadosamente cada consulta y desarrolla el método correspondiente dada la firma del método requerida.

##### Nota: Para poder trabajar con este notebook es necesario haber terminado el ejercicio de la sesión 06

In [1]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
%run utils.py

In [2]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as f
import pyspark.sql.types as t

# Creación de sesión de Spark
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ejercicio_7") \
    .getOrCreate()

spark.conf.set("spark.sql.session.timeZone", "GMT-6")

# Carga de tablas requeridas
root_path = "../resources/data/tmp/parquet/"
names_list = ["06/movies", "06/ratings", "06/tags"]
df_dict = read_tmp_df(spark, names_list)

movies_df = df_dict["06/movies"]
ratings_df = df_dict["06/ratings"]
tags_df = df_dict["06/tags"]

movies_df.show(1, False)
ratings_df.show(1)
tags_df.show(1)

+--------+----------------+-------------------------------------------------+----+
|movie_id|title           |genres                                           |year|
+--------+----------------+-------------------------------------------------+----+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1995|
+--------+----------------+-------------------------------------------------+----+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|rating|               time|
+-------+--------+------+-------------------+
|      1|       1|   4.0|2008-11-03 11:52:19|
+-------+--------+------+-------------------+
only showing top 1 row

+-------+--------+------+-------------------+
|user_id|movie_id|   tag|               time|
+-------+--------+------+-------------------+
| 224183|     832|acting|2017-06-05 07:20:27|
+-------+--------+------+-------------------+
only showing top 1 row



#### Actividad 1:
##### TO DO ->    Para el dataframe "movies_df":
- ##### Genera un método llamado get_all_genres que retorne un DataFrame con únicamente una columna conteniendo todos los valores distintos (sin repetir) de la columna "genres"
    - ##### Firma: def get_all_genres(df: DataFrame) -> DataFrame
    - Apoyate de la funcion explode -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html#pyspark.sql.functions.explode

- ##### Genera un método llamado get_min_year que retorne un valor de tipo int que contenga el menor año registrado (omitiendo nulls)
    - ##### Firma: def get_min_year(df: DataFrame) -> int
    - ##### Necesitarás llamar alguna de las siguientes acciones: take, first, head.

- ##### Genera un método llamado get_max_year que retorne un valor de tipo int que contenga el mayor año registrado (omitiendo nulls)
    - ##### Firma: def get_max_year(df: DataFrame) -> int
    - ##### Necesitarás llamar alguna de las siguientes acciones: take, first, head.

##### NO UTILIZAR withColumn NI withColumnRenamed

In [3]:
# TABLA del Data Frame movies_df:
movies_df.show(1, False)

+--------+----------------+-------------------------------------------------+----+
|movie_id|title           |genres                                           |year|
+--------+----------------+-------------------------------------------------+----+
|1       |Toy Story (1995)|[Adventure, Animation, Children, Comedy, Fantasy]|1995|
+--------+----------------+-------------------------------------------------+----+
only showing top 1 row



In [4]:
## TU CODIGO VA EN ESTA CELDA:
# modificar codigo interno
#def get_all_genres(df: DataFrame) -> DataFrame:
    #return df
def get_all_genres(df):
    gag_df = df\
        .select(
            f.explode(f.col("genres")).alias("list_of_genres")
        )\
        .distinct()
    return gag_df
    
#def get_min_year(df: DataFrame) -> int:
def get_min_year(df):
    gminy_df = df.groupBy(f.col("year"))\
    .agg(
        f.count("*").alias("count")
    )\
    .orderBy(f.col("year").asc(), f.col("count").asc())\
    .filter(f.col("year").isNotNull())
    return gminy_df.select("year").first()[0] # modificar codigo interno
    
#def get_max_year(df: DataFrame) -> int:
def get_max_year(df):
    gmaxy_df = df.groupBy(f.col("year"))\
    .agg(
        f.count("*").alias("count")
    )\
    .orderBy(f.col("year").desc(), f.col("count").desc())\
    .filter(f.col("year").isNotNull())
    return gmaxy_df.select("year").first()[0] # modificar codigo interno

In [5]:
# Elaboración de método:
"""
gag_df = movies_df\
        .select(
            f.explode(f.col("genres")).alias("list_of_genres")
        )\
        .distinct()
"""

'\ngag_df = movies_df        .select(\n            f.explode(f.col("genres")).alias("list_of_genres")\n        )        .distinct()\n'

In [6]:
# Elaboración de método:
"""
gminy_df = movies_df.groupBy(f.col("year"))\
    .agg(
        f.count("*").alias("count")
    )\
    .orderBy(f.col("year").asc(), f.col("count").asc())\
    .filter(f.col("year").isNotNull())
gminy_df.show()
gminy_df.select("year").first()[0]
"""

'\ngminy_df = movies_df.groupBy(f.col("year"))    .agg(\n        f.count("*").alias("count")\n    )    .orderBy(f.col("year").asc(), f.col("count").asc())    .filter(f.col("year").isNotNull())\ngminy_df.show()\ngminy_df.select("year").first()[0]\n'

In [7]:
# Elaboración de método:
"""
gmaxy_df = movies_df.groupBy(f.col("year"))\
    .agg(
        f.count("*").alias("count")
    )\
    .orderBy(f.col("year").desc(), f.col("count").desc())\
    .filter(f.col("year").isNotNull())
gmaxy_df.show()
gmaxy_df.select("year").first()[0]
"""

'\ngmaxy_df = movies_df.groupBy(f.col("year"))    .agg(\n        f.count("*").alias("count")\n    )    .orderBy(f.col("year").desc(), f.col("count").desc())    .filter(f.col("year").isNotNull())\ngmaxy_df.show()\ngmaxy_df.select("year").first()[0]\n'

In [8]:
# TEST:
"""
print("\tPrueba de método 1:\n")
get_all_genres(movies_df).show(20, False)
print("\tPrueba de método 2:\n")
print(get_min_year(movies_df))
print("\tPrueba de método 3:\n")
print(get_max_year(movies_df))
"""

'\nprint("\tPrueba de método 1:\n")\nget_all_genres(movies_df).show(20, False)\nprint("\tPrueba de método 2:\n")\nprint(get_min_year(movies_df))\nprint("\tPrueba de método 3:\n")\nprint(get_max_year(movies_df))\n'

In [9]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

get_all_genres(movies_df).show(20, False)
"""
Ejemplo de salida esperada (el nombre de la columna podría ser distinto):
+-----------+
|col        |
+-----------+
|Crime      |
|Romance    |
|Thriller   |
|Adventure  |
|Drama      |
|War        |
|Documentary|
|Fantasy    |
|Mystery    |
|Musical    |
|Animation  |
|Film-Noir  |
|IMAX       |
|Horror     |
|Western    |
|Comedy     |
|Children   |
|Action     |
|Sci-Fi     |
+-----------+
"""
print(get_min_year(movies_df))
# Salida esperada: 1874
print(get_max_year(movies_df))
# Salida esperada: 2023

+--------------+
|list_of_genres|
+--------------+
|Crime         |
|Romance       |
|Thriller      |
|Adventure     |
|Drama         |
|War           |
|Documentary   |
|Fantasy       |
|Mystery       |
|Musical       |
|Animation     |
|Film-Noir     |
|IMAX          |
|Horror        |
|Western       |
|Comedy        |
|Children      |
|Action        |
|Sci-Fi        |
+--------------+

1874
2023


In [10]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

genres_df = get_all_genres(movies_df)

assert type(genres_df) == DataFrame
assert len(genres_df.columns) == 1

genres_list = genres_df.rdd.map(lambda item: item[0]).collect()
genres_list.sort()
assert genres_list == ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 
                       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
                       'Thriller', 'War', 'Western']

min_year = get_min_year(movies_df)
assert min_year == 1874

max_year = get_max_year(movies_df)
assert max_year == 2023

#### Actividad 2:
##### TO DO -> Para la tabla "ratings_df" el cliente requiere hacer un análisis de cada "movie_id" para tener una idea de qué tan buena es cada pelicula, asi que nos solicitó desarrollar un método que realice múltiples cálculos.
##### Generar un método que retorne un DataFrame con las columnas especificadas por cada "movie_id":
- ##### Firma: def calculate_rating_values(df: DataFrame) -> DataFrame
- ##### Columnas generadas:
    - ##### nombre: avg_rating, tipo: DoubleType() -> Valor rating promedio (Redondear a 2 decimales)
    - ##### nombre: stddev_rating, tipo: DoubleType() -> Desviacion estándar (stddev_pop) para la columna rating (Redondear a 2 decimales)
        - Para redondear valores utiliza la función round de Spark -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.round.html#pyspark.sql.functions.round
    - ##### nombre: count_rating, tipo: LongType() -> Total de calificaciones recibidas
- ##### ACTUALIZACIÓN: Para aquellas peliculas (movie_id) que no se tenga identificado su año en la tabla "movies_df" nos han solicitado calcular el año de la siguiente manera (más adelante nos preocuparemos por pegar el año a la tabla movies_df):
    - ##### nombre: min_time_rating, tipo: TimestampType() -> Fecha más antigua de la columna "time" en la que se asignó el primer rating
##### Nota 1: podemos hacer el calculo de "min_time_rating" en la misma transformación en las que se generan las columnas "avg_rating", "stddev_rating", y "count_rating"
##### Nota 2: Posiblemente requieras analizar las funciones de agregación existentes en Spark -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#aggregate-functions
* El dataframe de salida deberá contar con la siguiente estructura al hacer printSchema():
* |-- movie_id: string
* |-- avg_rating: double
* |-- stddev_rating: double
* |-- count_rating: long
* |-- min_time_rating: timestamp
##### NO UTILIZAR withColumn NI withColumnRenamed

In [11]:
ratings_df.show(5)

+-------+--------+------+-------------------+
|user_id|movie_id|rating|               time|
+-------+--------+------+-------------------+
|      1|       1|   4.0|2008-11-03 11:52:19|
|      1|     110|   4.0|2008-11-05 00:04:46|
|      1|     158|   4.0|2008-11-03 11:31:43|
|      1|     260|   4.5|2008-11-03 12:00:04|
|      1|     356|   5.0|2008-11-03 11:58:39|
+-------+--------+------+-------------------+
only showing top 5 rows



In [12]:
movies_df.show(5, False)
movies_df.orderBy(f.col("year").asc()).show(5, False)

+--------+----------------------------------+-------------------------------------------------+----+
|movie_id|title                             |genres                                           |year|
+--------+----------------------------------+-------------------------------------------------+----+
|1       |Toy Story (1995)                  |[Adventure, Animation, Children, Comedy, Fantasy]|1995|
|2       |Jumanji (1995)                    |[Adventure, Children, Fantasy]                   |1995|
|3       |Grumpier Old Men (1995)           |[Comedy, Romance]                                |1995|
|4       |Waiting to Exhale (1995)          |[Comedy, Drama, Romance]                         |1995|
|5       |Father of the Bride Part II (1995)|[Comedy]                                         |1995|
+--------+----------------------------------+-------------------------------------------------+----+
only showing top 5 rows

+--------+--------------------------------------------------------

In [13]:
ratings_df.show(5)

+-------+--------+------+-------------------+
|user_id|movie_id|rating|               time|
+-------+--------+------+-------------------+
|      1|       1|   4.0|2008-11-03 11:52:19|
|      1|     110|   4.0|2008-11-05 00:04:46|
|      1|     158|   4.0|2008-11-03 11:31:43|
|      1|     260|   4.5|2008-11-03 12:00:04|
|      1|     356|   5.0|2008-11-03 11:58:39|
+-------+--------+------+-------------------+
only showing top 5 rows



In [14]:
## TU CODIGO VA EN ESTA CELDA:
# modificar codigo interno
#def calculate_rating_values(df: DataFrame) -> DataFrame:
def calculate_rating_values(df):
    crv_ratings_df = df.groupBy(f.col("movie_id"))\
    .agg(
        f.round(f.avg("rating"),2).alias("avg_rating"),
        f.round(f.stddev_pop("rating"),2).alias("stddev_rating"),
        f.count("rating").alias("count_rating"),
        f.min("time").alias("min_time_rating")
    )
    return crv_ratings_df

In [15]:
calculate_rating_values(ratings_df).show(2)
calculate_rating_values(ratings_df).printSchema()

rating_values_df = calculate_rating_values(ratings_df)
rating_values_df.count()

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|     296|      4.19|         0.95|      108756|1996-02-29 10:48:44|
|  115713|      3.99|         0.83|       21335|2015-01-02 06:05:51|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows

root
 |-- movie_id: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- stddev_rating: double (nullable = true)
 |-- count_rating: long (nullable = false)
 |-- min_time_rating: timestamp (nullable = true)



83239

In [16]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
calculate_rating_values(ratings_df).show(2)
"""
Ejemplo de salida esperada:
+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|     296|      4.19|         0.95|      108756|1996-02-29 10:48:44|
|  115713|      3.99|         0.83|       21335|2015-01-02 06:05:51|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows
"""

+--------+----------+-------------+------------+-------------------+
|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------+-------------+------------+-------------------+
|     296|      4.19|         0.95|      108756|1996-02-29 10:48:44|
|  115713|      3.99|         0.83|       21335|2015-01-02 06:05:51|
+--------+----------+-------------+------------+-------------------+
only showing top 2 rows



'\nEjemplo de salida esperada:\n+--------+----------+-------------+------------+-------------------+\n|movie_id|avg_rating|stddev_rating|count_rating|    min_time_rating|\n+--------+----------+-------------+------------+-------------------+\n|     296|      4.19|         0.95|      108756|1996-02-29 10:48:44|\n|  115713|      3.99|         0.83|       21335|2015-01-02 06:05:51|\n+--------+----------+-------------+------------+-------------------+\nonly showing top 2 rows\n'

In [17]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA

rating_values_df = calculate_rating_values(ratings_df)

assert type(rating_values_df) == DataFrame
assert len(rating_values_df.columns) == 5
assert rating_values_df.count() == 83239

data = [Row("296", 4.19, 0.95, 108756, datetime.datetime(1996, 2, 29, 10, 48, 44))]

schema = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("avg_rating", t.DoubleType()),
    t.StructField("stddev_rating", t.DoubleType()),
    t.StructField("count_rating", t.LongType()),
    t.StructField("min_time_rating", t.TimestampType())])

test_df = spark.createDataFrame(data, schema)
assert rating_values_df.filter(f.col("movie_id") == "296").subtract(test_df).count() == 0

#### Actividad 3:
##### TO DO -> El cliente ha solicitado generar dos tablas con el mismo contenido pero con distinto esquema a partir de la tabla "tags_df", lee a continuación la información enviada con los requerimientos:
- ##### En ambas tablas se requiere agrupar por "movie_id" y calcular:
    - ##### El total de veces en las que aparece cada tag (en mayusculas) .
    - ##### Fecha más antigua en la que se asignó el primer tag.
La primer tabla deberá tener la siguiente estructura:  
*      | movie_id |             tag_count |        min_time_tag |
*      |        1 | [SCI-FI:2, TERROR:12] | 2017-06-02 07:20:27 |
*      |        3 |  [DRAMA:14, SCI-FI:4] | 2012-06-02 07:20:27 |
con esquema:
*      |-- movie_id: string
*      |-- tag_count: array
*      |    |-- element: string
*      |-- min_time_tag: timestamp
La segunda tabla deberá tener la siguiente estructura:
*      | movie_id |                   tag_count |        min_time_tag |
*      |        1 | [{SCI-FI, 2}, {TERROR, 12}] | 2017-06-02 07:20:27 |
*      |        3 |  [{DRAMA, 14}, {SCI-FI, 4}] | 2012-06-02 07:20:27 |
con esquema:
*      |-- movie_id: string
*      |-- tag_count: array
*      |    |-- element: struct
*      |    |    |-- tag: string
*      |    |    |-- count: long
*      |-- min_time_tag: timestamp
- ##### NOTA: Ordena la columna tag_count, la cual es un array, de forma ascendente.
- #### La generación de la primer tabla es a través del método con la firma:
    - ##### def get_act_3_df1(df: DataFrame) -> DataFrame
- #### La generación de la segunda tabla es a través del método con la firma:
    - ##### def get_act_3_df2(df: DataFrame) -> DataFrame

#### La siguiente información muestra los pasos recomendados para resolver la actividad 3, puedes omitir leer esta parte si asi lo consideras.
##### Para generar ambas tablas primero necesitamos obtener el total veces (count) en las que un "tag" se repite por cada "movie_id", para evitar conteos erroneos hay que convertir cada tag en Mayusculas.
##### El dataframe resultante deberá contener la siguiente estructura: "movie_id", "tag", "count" y "min" donde la columna "count" representa el total de veces que aparecen cada "tag"y "movie_id"; la columna "min" representa el valor mínimo de cada "tag" y "movie_id".
Por ejemplo, dado el dataframe de entrada
*      |user_id|movie_id|   tag|               time|
*      |    183|     100|sci-fi|2012-06-02 07:20:27|
*      |     12|     832| Drama|2017-06-01 07:20:27|
*      |    251|     100|SCI-FI|2009-06-04 07:20:27|
*      |    265|     832| DRAMA|2015-06-08 07:20:27|
*      |     22|     100|terror|2020-06-06 07:20:27|

debemos obtener el siguiente dataframe (el nombre de columnas "count" y "min" podria ser distinto):

*      | movie_id |    tag | count |                 min |
*      |      100 | TERROR |     1 | 2020-06-06 07:20:27 |
*      |      100 | SCI-FI |     2 | 2009-06-04 07:20:27 |
*      |      832 |  DRAMA |     2 | 2015-06-08 07:20:27 |

##### Para generar la primer tabla necesitamos concatenar las columnas "tag" y "count", posteriormente agrupando por "movie_id" generaremos la lista de elementos requerida. La columna "min_time_tag" representa el valor mínimo de cada "movie_id".
- ##### Funciones de Spark recomendadas:
    - upper -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.upper.html#pyspark.sql.functions.upper
    - concat_ws -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.concat_ws.html#pyspark.sql.functions.concat_ws
    - concat -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.concat.html#pyspark.sql.functions.concat
    - collect_list -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.collect_list.html#pyspark.sql.functions.collect_list
    - collect_set -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.collect_set.html#pyspark.sql.functions.collect_set##### 
    - sort_array -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sort_array.html#pyspark-sql-functions-sort-array
##### La primer tabla quedará estructurada de la siguiente manera, donde la columna "tag_count" es de tipo ArrayType(StringType()) y la columna "min_time_tag" es de tipo TimestampType():
Dado el dataframe de entrada
*      | movie_id |    tag | count |                 min |
*      |        1 | TERROR |    12 | 2020-06-06 07:20:27 |
*      |        1 | SCI-FI |     2 | 2015-06-06 07:20:27 |
*      |        3 |  DRAMA |    14 | 2004-06-06 07:20:27 |
*      |        3 | SCI-FI |     4 | 2012-06-06 07:20:27 |
debemos obtener el siguiente dataframe
*       | movie_id |             tag_count |        min_time_tag |
*       |        1 | [SCI-FI:2, TERROR:12] | 2015-06-06 07:20:27 |
*       |        3 |  [DRAMA:14, SCI-FI:4] | 2004-06-06 07:20:27 |
##### Para generar la segunda tabla necesitamos agregar en una estructura las columnas "tag" y "count", posteriormente agrupando por movie_id generaremos la lista de elementos requerida. La columna "min_time_tag" representa el valor mínimo de cada "movie_id".
- ##### Funciones de Spark recomendadas:
    - upper -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.upper.html#pyspark.sql.functions.upper
    - struct -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.struct.html#pyspark.sql.functions.struct
    - collect_list -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.collect_list.html#pyspark.sql.functions.collect_list
    - collect_set -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.collect_set.html#pyspark.sql.functions.collect_set
    - sort_array -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sort_array.html#pyspark-sql-functions-sort-array
##### La segunda tabla quedará estructurada de la siguiente manera, donde la columna "tag_count" de tipo ArrayType(StructType(StringType(),LongType()))  y la columna "min_time_tag" es de tipo TimestampType():
Dado el dataframe de entrada
*      | movie_id |    tag | count |                 min |
*      |        1 | TERROR |    12 | 2020-06-06 07:20:27 |
*      |        1 | SCI-FI |     2 | 2015-06-06 07:20:27 |
*      |        3 |  DRAMA |    14 | 2004-06-06 07:20:27 |
*      |        3 | SCI-FI |     4 | 2012-06-06 07:20:27 |
debemos obtener el siguiente dataframe
*      | movie_id |                   tag_count |        min_time_tag |
*      |        1 | [{SCI-FI, 2}, {TERROR, 12}] | 2015-06-06 07:20:27 |
*      |        3 |  [{DRAMA, 14}, {SCI-FI, 4}] | 2004-06-06 07:20:27 |

In [18]:
# Tabla con la que trabajaremos:
tags_df.show(10,False)

+-------+--------+--------------------+-------------------+
|user_id|movie_id|tag                 |time               |
+-------+--------+--------------------+-------------------+
|224183 |832     |acting              |2017-06-05 07:20:27|
|224183 |832     |action              |2017-06-05 07:20:01|
|224183 |832     |Brawley Nolte       |2017-06-05 07:19:31|
|224183 |832     |conspiracy theory   |2017-06-05 07:20:00|
|224183 |832     |Director: Ron Howard|2017-06-05 07:15:05|
|224183 |832     |Donnie Wahlberg     |2017-06-05 07:19:33|
|224183 |832     |Gary Sinise         |2017-06-05 07:19:30|
|224183 |832     |kidnapping          |2017-06-05 07:19:57|
|224183 |832     |Liev Schreiber      |2017-06-05 07:19:36|
|224183 |832     |Lili Taylor         |2017-06-05 07:19:28|
+-------+--------+--------------------+-------------------+
only showing top 10 rows



In [19]:
## TU CODIGO VA EN ESTA CELDA, PUEDES GENERAR MÉTODOS O VARIABLES NUEVAS SI ASI LO REQUIERES

#def get_act_3_df1(df: DataFrame):
# ... transformaciones a tags_df
def get_act_3_df1(df):
    df_mtt = df.groupBy(f.col("movie_id"), f.upper(f.col("tag")).alias("tag"))\
        .agg(
            f.count(f.col("tag")).alias("count"),
            f.min("time").alias("min")
        ).select(
            f.col("movie_id"),
            f.concat_ws(" : ", f.col("tag"),f.col("count")).alias("tag_count"),
            f.col("min").alias("min_tag")
        )
    tabla1_df = df_mtt.groupBy(f.col("movie_id"))\
        .agg(
            f.sort_array(f.collect_set("tag_count")).alias("tag_count"),
            f.min("min_tag").alias("min_time_tag"),
        )
    return tabla1_df

# ... transformaciones a tags_df
def get_act_3_df2(df):
    df_mtt2 = df.groupBy(f.col("movie_id"), f.upper(f.col("tag")).alias("tag"))\
        .agg(
            f.count(f.col("tag")).alias("count"),
            f.min("time").alias("min")
        ).select(
            f.col("movie_id"),
            f.struct(f.col("tag"),f.col("count")).alias("struct"),
            f.col("min").alias("min_tag")
        )
    tabla2_df = df_mtt2.groupBy(f.col("movie_id"))\
        .agg(
            f.sort_array(f.collect_set("struct")).alias("tag_count"),
            f.min("min_tag").alias("min_time_tag")
        )
    return tabla2_df

act_3_df1 = get_act_3_df1(tags_df)
act_3_df2 = get_act_3_df2(tags_df)

act_3_df1.show(2,False)
act_3_df1.printSchema()

act_3_df2.show(2,False)
act_3_df2.printSchema()

+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

root
 |-- movie_id: string (nullable = true)
 |-- tag_count: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- min_time_tag: timestamp (nullable = true)

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |


In [20]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
act_3_df1.show(2, False)
act_3_df2.show(2, False)
"""
Ejemplo de salida esperada:
+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{FATE, 1}, {PRESS-GANGED, 1}, {WAR, 1}, {WORLD WAR II, 1}]           |2018-05-26 16:40:54|
|100070  |[{COMEDIAN, 2}, {COMEDY, 1}, {GOOD HUMOUR, 1}, {STRUGGLING CAREER, 1}]|2017-05-19 17:17:36|
+--------+----------------------------------------------------------------------+-------------------+
only showing top 2 rows
"""

+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

+--------+----------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                             |min_time_tag       |
+--------+----------------------------------------------------------------------+-------------------+
|100062  |[{FATE, 1}, {PRESS-GANGED, 1}, {WAR, 1}, {WORLD WAR II, 1}]           |

'\nEjemplo de salida esperada:\n+--------+------------------------------------------------------------------+-------------------+\n|movie_id|tag_count                                                         |min_time_tag       |\n+--------+------------------------------------------------------------------+-------------------+\n|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|\n|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|\n+--------+------------------------------------------------------------------+-------------------+\nonly showing top 2 rows\n\n+--------+----------------------------------------------------------------------+-------------------+\n|movie_id|tag_count                                                             |min_time_tag       |\n+--------+----------------------------------------------------------------------+-------------------+\n|100062  |[{FATE, 1}, {PRESS-GANGED, 1

In [21]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
expected_df1 = [
    Row(movie_id='100070',
        tag_count=['COMEDIAN : 2', 'COMEDY : 1', 'GOOD HUMOUR : 1', 'STRUGGLING CAREER : 1'],
        min_time_tag=datetime.datetime(2017, 5, 19, 17, 17, 36))]
schema_df1 = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("tag_count", t.ArrayType(t.StringType())),
    t.StructField("min_time_tag", t.TimestampType())
])
assert len(act_3_df1.columns) == 3
assert "movie_id" in act_3_df1.columns
assert "tag_count" in act_3_df1.columns
assert "min_time_tag" in act_3_df1.columns
assert schema_to_ddl(spark, act_3_df1.select("movie_id", "tag_count", "min_time_tag")) == 'movie_id STRING,tag_count ARRAY<STRING>,min_time_tag TIMESTAMP'
assert act_3_df1.count() == 53452
assert act_3_df1.filter(f.col("movie_id") == "100070").subtract(spark.createDataFrame(expected_df1, schema_df1)).count() == 0

expected_df2 = [
    Row(movie_id='100070',
        tag_count=[Row(tag='COMEDIAN', count=2),
                   Row(tag='COMEDY', count=1),
                   Row(tag='GOOD HUMOUR', count=1),
                   Row(tag='STRUGGLING CAREER', count=1)],
        min_time_tag=datetime.datetime(2017, 5, 19, 17, 17, 36))]
schema_df2 = t.StructType([
    t.StructField("movie_id", t.StringType()),
    t.StructField("tag_count", t.ArrayType(t.StructType([
        t.StructField("tag", t.StringType()),
        t.StructField("count", t.LongType())
    ]))),
    t.StructField("min_time_tag", t.TimestampType())
])
assert len(act_3_df2.columns) == 3
assert "movie_id" in act_3_df2.columns
assert "tag_count" in act_3_df2.columns
assert "min_time_tag" in act_3_df2.columns
assert schema_to_ddl(spark, act_3_df2.select("movie_id", "tag_count", "min_time_tag")) == 'movie_id STRING,tag_count ARRAY<STRUCT<tag: STRING, count: BIGINT>>,min_time_tag TIMESTAMP'
assert act_3_df2.count() == 53452
assert act_3_df2.filter(f.col("movie_id") == "100070").subtract(spark.createDataFrame(expected_df2, schema_df2)).count() == 0

#### Actividad 4:
##### TO DO -> El cliente ha solicitado que resolvamos dos consultas con la salida de la actividad 3, como no especificó con cuál dataframe requiere la consulta lo realizaremos con ambos dataframes (act_3_df1 y act_3_df2).
##### Las consultas a resolver son:
- ##### 1.- ¿Cuál es la pelicula (movie_id) con más tags con el valor "SCI-FI"?
    - ##### Estrictamente especificó no tomar en cuenta datos como "REALISTIC SCI-FI", "HARD SCI-FI", etc.
- ##### 2.- ¿Cuántas peliculas fueron etiquetadas como "SCI-FI"?
##### La forma de resolver estas consultas será a través de un método el cual va a retornar una tupla (str, int), donde el primer elemento (str) representa el resultado de la consulta 1 y el segundo elemento (int) representa el resultado de la consulta 2
- ##### La firma del método que utilizará cada dataFrame es:
    - ##### def exercise_4_df1(df: DataFrame) -> firma para el dataFrame act_3_df1
    - ##### def exercise_4_df2(df: DataFrame) -> firma para el dataFrame act_3_df2
##### Funciones de Spark recomendadas:
- explode -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html#pyspark-sql-functions-explode
- regexp_extract -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.regexp_extract.html#pyspark.sql.functions.regexp_extract
##### Funciones de la clase Column recomendadas:
- like -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.like.html#pyspark.sql.Column.like
- ilike -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.ilike.html#pyspark.sql.Column.ilike
- getField -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.getField.html#pyspark.sql.Column.getField
##### Acciones que podrias utilizar:
- count, first, head, take

In [22]:
act_3_df1.show(2, False)
act_3_df1.printSchema()

"""ex4_op = act_3_df1\
    .select(
        f.col("movie_id"),
        f.explode(f.col("tag_count")).alias("key"),
        f.regexp_extract(f.col("key"),'SCI-FI\s*:\s*(\d+)',0).alias("sci-fi_count")
    ).orderBy(f.col("sci-fi_count").desc())

ex4_op.show(5,False)"""

+--------+------------------------------------------------------------------+-------------------+
|movie_id|tag_count                                                         |min_time_tag       |
+--------+------------------------------------------------------------------+-------------------+
|100062  |[FATE : 1, PRESS-GANGED : 1, WAR : 1, WORLD WAR II : 1]           |2018-05-26 16:40:54|
|100070  |[COMEDIAN : 2, COMEDY : 1, GOOD HUMOUR : 1, STRUGGLING CAREER : 1]|2017-05-19 17:17:36|
+--------+------------------------------------------------------------------+-------------------+
only showing top 2 rows

root
 |-- movie_id: string (nullable = true)
 |-- tag_count: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- min_time_tag: timestamp (nullable = true)



'ex4_op = act_3_df1    .select(\n        f.col("movie_id"),\n        f.explode(f.col("tag_count")).alias("key"),\n        f.regexp_extract(f.col("key"),\'SCI-FI\\s*:\\s*(\\d+)\',0).alias("sci-fi_count")\n    ).orderBy(f.col("sci-fi_count").desc())\n\nex4_op.show(5,False)'

In [23]:
## TU CODIGO VA EN ESTA CELDA, PUEDES GENERAR MÉTODOS O VARIABLES SI ASI LO REQUIERES

#Para el data frame act_3_df1:
def exercise_4_df1(df):
    ex4_df1 = df\
        .select(
            *df.columns,
            f.explode(f.col("tag_count")).alias("tag")
        )\
        .select(
            *df.columns,
            f.col("tag"),
            f.regexp_extract(f.col("tag"), "[0-9]*$", 0).cast(t.IntegerType()).alias("tot")
        )\
        .filter(f.col("tag").like("SCI-FI : %"))\
        .orderBy(f.col("tot").desc())\
        #.select("movie_id").first()[0]
    #ex4_df1_2 = df
    return (ex4_df1.select("movie_id").first()[0],ex4_df1.select("movie_id").distinct().count()) # modificar codigo interno

#Para el data frame act_3_df2:
def exercise_4_df2(df):
    ex4_df2 = df\
        .select(
            *df.columns,
            f.explode(f.col("tag_count")).alias("tag")
        ).\
        select(
            *df.columns,
            f.col("tag").getField("tag"),
            f.col("tag").getField("count") #regexp_extract trabaja con string y no con struct
        )\
        .filter(f.col("tag.tag").like("SCI-FI"))\
        .orderBy(f.col("tag.count").desc())
    return (ex4_df2.select("movie_id").first()[0],ex4_df2.select("movie_id").distinct().count()) # modificar codigo interno

In [24]:
print(exercise_4_df1(act_3_df1))
print(exercise_4_df2(act_3_df2))

('260', 854)
('260', 854)


In [25]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
print(exercise_4_df1(act_3_df1))
print(exercise_4_df2(act_3_df2))
# Salida esperada en ambos casos
#('260', 854)

('260', 854)
('260', 854)


In [26]:
movie_id, count = exercise_4_df1(act_3_df1)
assert movie_id == "260"
assert count == 854

movie_id, count = exercise_4_df2(act_3_df2)
assert movie_id == "260"
assert count == 854

In [27]:
# NO MODIFICAR EL CONTENIDO DE ESTA CELDA
dfs = [(movies_df, "07/movies"),
       (act_3_df1, "07/tags_p1"),
       (act_3_df2, "07/tags_p2"),
       (calculate_rating_values(ratings_df), "07/ratings")]

write_tmp_df(dfs)